### Import Libraries

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
import numpy as np

### Load Data and preprocess

In [2]:
RatingsData = pd.read_csv("datasets/XWines_Full_21M_ratings.csv")

RatingsData = RatingsData.drop("Date",axis=1)
RatingsData = RatingsData.drop("RatingID",axis=1)

RatingsData=RatingsData.drop_duplicates(subset=['UserID','WineID'])
df = pd.DataFrame(RatingsData.groupby("WineID")["UserID"].count().sort_values(ascending=False).head(100))
df = df.merge(RatingsData,left_on="WineID",right_on="WineID")
RatingsData = df.drop("UserID_x",axis=1)
RatingsData= RatingsData.rename(columns={"UserID_y": "UserID"})
df = pd.DataFrame(RatingsData.groupby("UserID")["WineID"].count().sort_values(ascending=False).head(1000))
df = df.merge(RatingsData,left_on="UserID",right_on="UserID")
RatingsData = df.drop("WineID_x",axis=1)
RatingsData= RatingsData.rename(columns={"WineID_y": "WineID"})
RatingsDataPerWine=pd.pivot(RatingsData,index=["UserID"],columns=["WineID"],values="Rating")
RatingsDataPerWine = RatingsDataPerWine.replace({np.nan: None})
RatingsDataPerWine.to_csv("datasets/RatingsPerWine.csv")

C:\Users\P2001\AppData\Local\Temp\ipykernel_22088\2186090489.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  RatingsData = pd.read_csv("datasets/XWines_Full_21M_ratings.csv")


### Utility Functions

In [3]:
def dot(u,v):
    return sum([u[i]*v[i] for i in range(len(u)) if u[i] is not None and v[i] is not None])

def length(u, v):
    return (sum([u[i]*u[i] for i in range(len(u)) if u[i] is not None and v[i] is not None]))**0.5

def bias(u):
    u_not_none = [u[i] for i in range(len(u)) if u[i] is not None]
    return sum(u_not_none)/len(u_not_none)

def bias_a_vector(u):
    b = bias(u)
    return [u[i]-b if u[i] is not None else None for i in range(len(u))]

def sim(u,v, bias=False, Dataset=RatingsDataPerWine):
    if u>v:
        return np.nan
    u=Dataset.iloc[u].to_list()
    v=Dataset.iloc[v].to_list()
    if not bias:
        return dot(u, v)/(length(u, v)*length(v, u))
    else:
        if (length(bias_a_vector(u), v)*length(bias_a_vector(v), u))==0 :
            return 0
        else :
            return dot(bias_a_vector(u), bias_a_vector(v))/(length(bias_a_vector(u), v)*length(bias_a_vector(v), u))

### Similarity test

In [4]:
lenData= len(RatingsDataPerWine)
matUserSim=[]
for z in range(lenData):
    matUserSim.append([sim(z,j,bias=True) for j in range(lenData)])

df= pd.DataFrame(matUserSim)
df.to_csv("datasets/test.csv",index=False)